Тематическое моделирование на своем датасете: использование модели LDA, визуализация топиков, построение тематических профилей.
Цель: Боевой проект по тематическому моделированию. Цель - построить LDA модель и получить интерпретируемые топики. Попробовать посчитать тематические профили и получить их визуализацию на двумерной плоскости при помощи TSNE
В материалах вы можете найти выгрузку комментариев из разных музыкальных групп ВК, в полях music_style и performer можно получить информацию о конкретном музыкальном жанре и исполнителе, в группе которого был написан комментарий. Выгрузка уменьшенная, остались только комментарии длиной больше 40 слов, но тем не менее это всё ещё 500Мб, так что модельки будут считаться долго. Для отладки рекомендую брать небольшую выборку из этого датасета, чтобы на ней тестировать свой код.

Если у вас есть собственный датасет - можно (и будет очень круто) использовать его :)

1. Загрузите датасет. Если используете данные вк, то после загрузки воспользуйтесь следующим кодом:

from ast import literal_eval
bag_of_words = data.text_bow.apply(literal_eval)

где data - это загруженный датасет. Операция необходима, так как предварительно уже была проведена предобработка текстов и получено представление bag-of-words, так что остаётся его лишь прочитать из датасета

2. Используя gensim, добавьте биграммы к имеющемуся bag_of_words представлению

3. Составьте словарь из терминов (corpora.Dictionary)

4. По словарю терминов при помощи метода filter_extremes проведите фильтрацию слов по частоте встречаемости. На данных вк рекомендую следующие параметры: (no_below=3, no_above=0.4, keep_n=3*10**6)

5. Наконец, составьте итоговый корпус документов при помощи метода doc2bow. Рекомендую его сохранить (например, в pickle формате), на случай, если что-то где-то пойдет не так

6. По получившемуся корпусу постройте LDA модель, если используете данные вк, рекомендую параметры (eval_every=20, num_topics=30, passes=5). Не забудьте сохранить саму модель и её аттрибут expElogBeta, также при помощи метода show_topics возьмите по 100 самых вероятных слов для каждой темы

7. При помощи wordcloud визуализируйте каждый из получившихся топиков и попробуйте их проинтерпретировать. Чтобы не возиться долго с функциями для визуализации, можно взять их отсюда - https://github.com/DmitrySerg/top-russian-music/blob/master/models/clean_visualization.ipynb

8. Наконец, постройте тематические профили по интересующим вас группировкам (в случае с вк можно построить средние тематические вектора по жанрам или по исполнителям, если у вас собственные данные, попробуйте подумать, как можно было бы сгруппировать ваши документы, чтобы все-таки построить профили групп), и визуализируйте их при помощи TSNE

9. Вы великолепны
Критерии оценки: Получилось построить LDA и хотя бы в текстовом формате показать наиболее характерные для каждого топика слова - 5 баллов

Есть визуализация wordcloud +2 балла

Есть визуализация TSNE для тематических профилей +3 балла


In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import json

from sklearn.preprocessing import LabelEncoder

Пару занятий назад спарсил местный сайт новостей, его и буду мучить. Цель - по тексту новости найти темы и сравнить их с реальными рубриками.

In [2]:
data = pd.read_csv('ngsnews.csv', sep=';')

In [3]:
data.head()

,Unnamed: 0,parced,link,datenews,headline,altheadline,txtnewsfull,views,comments,rating,rubric,tags
0,0,2020-02-08 13:52:56,https://ngs24.ru/news/more/66485668/,2020-02-08 12:42:26,Красноярская активистка направила обращение в ...,Ответ аппарата правительства ее рассмешил,Лаборатория должна появиться в горах в 6 км от...,703,9,ЛАЙК1СМЕХ1УДИВЛЕНИЕ0ГНЕВ0ПЕЧАЛЬ0,Город,"#Наталия Подоляк, #ядерное хранилище, #Краснод..."
1,1,2020-02-08 13:52:56,https://ngs24.ru/news/more/66485626/,2020-02-08 11:32:39,Молодые красноярцы сняли квартиру и оборудовал...,Им грозит 20 лет колонии,Найденная коробка со всякими веществами и один...,1863,4,ЛАЙК0СМЕХ0УДИВЛЕНИЕ0ГНЕВ0ПЕЧАЛЬ0,Криминал,"#нарколаборатория,"
2,2,2020-02-08 13:52:58,https://ngs24.ru/news/more/66485617/,2020-02-08 10:42:55,С ж/д моста на Свободном оторвалась балка и на...,На месте работают сотрудники ГИБДД,Балка опасно повисла над проезжей частью Фото:...,1905,7,ЛАЙК0СМЕХ0УДИВЛЕНИЕ1ГНЕВ2ПЕЧАЛЬ0,Происшествия,"#балка, #ЧП,"
3,3,2020-02-08 13:52:59,https://ngs24.ru/news/more/66485605/,2020-02-08 10:23:29,«Это грубое вторжение человека в экологию»: же...,Она предложила лечить его гомеопатией,Наталья Толоконская — врач в Новосибирском гом...,1192,16,ЛАЙК2СМЕХ0УДИВЛЕНИЕ0ГНЕВ0ПЕЧАЛЬ0,Здоровье Китайский коронавирус,"#гомеопатия, #Наталья Толоконская,"
4,4,2020-02-08 13:53:01,https://ngs24.ru/news/more/66485599/,2020-02-08 09:34:17,Красноярец прошелся по капоту припаркованного ...,Машина стояла припаркованной вплотную к стене,Задуманное молодому человеку удалось осуществи...,3181,14,ЛАЙК8СМЕХ3УДИВЛЕНИЕ1ГНЕВ2ПЕЧАЛЬ1,Дороги и транспорт,"#парковка,"


 собираем  текст новости  с  заголовком, подзаголовком  и собственно текстом  и очищаем от пунктуации, переводим в нижний регистр

In [4]:
import re
def strip_punctuation(string):
    return re.sub(r'[^\w\s]', '', string.lower())

data['text'] = data['headline'].fillna(" ") + ' ' + data['altheadline'].fillna(" ") + ' ' + data['txtnewsfull'].fillna(" ")
data['text'] = data['text'].apply(strip_punctuation)

print(data.text)
data.head(1).T


0      красноярская активистка направила обращение в ...
1      молодые красноярцы сняли квартиру и оборудовал...
2      с жд моста на свободном оторвалась балка и нав...
3      это грубое вторжение человека в экологию жена ...
4      красноярец прошелся по капоту припаркованного ...
5      под казанью на частном вертолете разбился депу...
6      спасатели показали как вытаскивают со дна зато...
7      чаще всего жертвами становятся молодые мамы ин...
8      суд присяжных оправдал парня убившего свою баб...
9      в бсмп прошли учения по приемке пациентов с ко...
10     вторые в мире по загрязнению вчерашний смог вы...
11     запрет youtube угроза штрафа за датчики и увол...
12     на младенца упал баллон с гелием девочка борол...
13     плати или 12 лет по делу эксдепутата аркадия в...
14     попытался убить и обокрал дивногорца приговори...
15     80летний пенсионер из красноярска устраивает м...
16     красноярских детей начали закалять от орви йог...
17     главный прокурор края ух

,0
Unnamed: 0,0
parced,2020-02-08 13:52:56
link,https://ngs24.ru/news/more/66485668/
datenews,2020-02-08 12:42:26
headline,Красноярская активистка направила обращение в ...
altheadline,Ответ аппарата правительства ее рассмешил
txtnewsfull,Лаборатория должна появиться в горах в 6 км от...
views,703
comments,9
rating,ЛАЙК1СМЕХ1УДИВЛЕНИЕ0ГНЕВ0ПЕЧАЛЬ0


Убираем стоп слова

In [5]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('russian')

def data_Stop(text):
    return ' '.join([word for word in text.split(' ') if word not in stop])

data['text'] = data['text'].apply(lambda x: data_Stop(x))
print(data.text)

0      красноярская активистка направила обращение пр...
1      молодые красноярцы сняли квартиру оборудовали ...
2      жд моста свободном оторвалась балка нависла до...
3      это грубое вторжение человека экологию жена эк...
4      красноярец прошелся капоту припаркованного авт...
5      казанью частном вертолете разбился депутат гос...
6      спасатели показали вытаскивают дна затонувшие ...
7      чаще жертвами становятся молодые мамы интервью...
8      суд присяжных оправдал парня убившего бабушку ...
9      бсмп прошли учения приемке пациентов коронавир...
10     вторые мире загрязнению вчерашний смог вывел к...
11     запрет youtube угроза штрафа датчики увольнени...
12     младенца упал баллон гелием девочка боролась ж...
13     плати 12 лет делу эксдепутата аркадия волкова ...
14     попытался убить обокрал дивногорца приговорили...
15     80летний пенсионер красноярска устраивает марш...
16     красноярских детей начали закалять орви йогой ...
17     главный прокурор края ух

Лемматизация и токенизация

In [6]:
from pymystem3 import Mystem
m = Mystem()
# применяем лемматизацию
data['text_bow'] = data['text'].apply(lambda x: m.lemmatize(x))
data.head(1).T
# заодно посчитаем длину новости
data['comment_len'] = data['text_bow'].apply(len)

Биграммы

In [8]:
from gensim import corpora, models
# создаём объект для генерации биграм
# min_count и threshold нужны для обрезки слишком редких
bigram = models.Phrases(data.text_bow, min_count=3, threshold=5)
# оборачиваем объект для быстрой обработки текста
bigram_mod = models.phrases.Phraser(bigram)
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
# и генерируем текст с биграммами
texts = make_bigrams(data.text_bow)

Фильтрация по частоте

In [9]:
from gensim import corpora, models
# составляем словарь из терминов
dictionary = corpora.Dictionary(texts)
# и убираем слишком редкие (no_below) и слишком частые (no_above)
dictionary.filter_extremes(no_below=3, no_above=0.4)
# наконец, составляем финальный готовый корпус
corpus = [dictionary.doc2bow(text) for text in texts]
# и не забываем сохранить
import pickle
with open('corpus', 'wb') as f:
    pickle.dump(corpus, f)

Построение модели

In [10]:
from gensim import corpora, models

ldamodel = models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=30,
    passes=5
    )

ldamodel.save("ldamodel_30_bigrams_long_comments")

topics = ldamodel.show_topics(
    num_topics=30,
    num_words=100,
    formatted=False
    )

with open('lda_30_topics_bigrams_long_comments', 'wb') as f:
    pickle.dump(topics, f)